In [2]:
import os
import argparse
import torch
import matplotlib.pyplot as plt
from tqdm import trange

# Módulos customizados – certifique-se de que eles estejam disponíveis no seu ambiente
from plot_utils import *
from utils import set_seed, cal_subtb_coef_matrix, fig_to_image, get_gfn_optimizer, get_gfn_forward_loss, \
    get_gfn_backward_loss, get_exploration_std, get_name
from buffer import ReplayBuffer
from langevin import langevin_dynamics
from models import GFN
from gflownet_losses import *
from energies import *
from evaluations import *


## Configuração dos Hiperparâmetros

Em notebooks é comum definirmos os parâmetros manualmente em vez de usar *argparse*.  
A seguir, criamos um objeto `args` (do tipo `Namespace`) com os valores padrão.


In [4]:
# Em vez de usar argparse.parse_args(), definimos os parâmetros manualmente:
args = argparse.Namespace(
    lr_policy=1e-3,
    lr_flow=1e-2,
    lr_back=1e-3,
    hidden_dim=64,
    s_emb_dim=64,
    t_emb_dim=64,
    harmonics_dim=64,
    batch_size=300,
    buffer_size=300 * 100 * 2,
    T=100,
    epochs=25000,
    subtb_lambda=2,
    t_scale=5.0,
    log_var_range=4.0,
    energy='linreg',  # opções: 'vae' ou 'linreg'
    mode_fwd="tb",  # opções: 'tb', 'tb-avg', 'db', 'subtb', 'cond-tb-avg'
    mode_bwd="tb",  # opções: 'tb', 'tb-avg', 'mle', 'cond-tb-avg'
    both_ways=False,
    repeats=10,
    local_search=False,
    max_iter_ls=200,
    burn_in=100,
    ls_cycle=100,
    ld_step=0.001,
    ld_beta=5.0,
    ld_schedule=False,
    target_acceptance_rate=0.574,
    beta=1.0,
    rank_weight=1e-2,
    prioritized="rank",  # opções: 'none', 'reward', 'rank'
    scheduler=False,
    step_point=7000,
    bwd=False,
    exploratory=False,
    sampling="buffer",  # opções: 'sleep_phase', 'energy', 'buffer'
    langevin=False,
    langevin_scaling_per_dimension=False,
    conditional_flow_model=False,
    learn_pb=False,
    pb_scale_range=0.1,
    learned_variance=False,
    partial_energy=False,
    exploration_factor=0.1,
    exploration_wd=False,
    clipping=False,
    lgv_clip=1e2,
    gfn_clip=1e4,
    zero_init=False,
    pis_architectures=False,
    lgv_layers=3,
    joint_layers=2,
    seed=12345,
    weight_decay=1e-7,
    use_weight_decay=False,
    eval=False
)

# Configuração adicional
eval_data_size = 300
final_eval_data_size = 300
plot_data_size = 16
final_plot_data_size = 16

if args.pis_architectures:
    args.zero_init = True


## Configuração do Ambiente

Nesta célula, configuramos a semente para reprodutibilidade, o dispositivo de computação e a matriz de coeficientes.


In [5]:
set_seed(args.seed)
if 'SLURM_PROCID' in os.environ:
    args.seed += int(os.environ["SLURM_PROCID"])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
coeff_matrix = cal_subtb_coef_matrix(args.subtb_lambda, args.T).to(device)

# Se ambos os modos (forward e backward) estiverem ativos, desabilita o modo backward
if args.both_ways and args.bwd:
    args.bwd = False

# Se a busca local estiver ativa, força ambos os modos
if args.local_search:
    args.both_ways = True

## Definição das Funções de Apoio

Nesta seção definimos as funções que compõem o pipeline de treinamento:

- **get_energy():** Inicializa o modelo de energia de acordo com o parâmetro `args.energy`.
- **plot_step():** Gera e salva figuras com os dados reais, amostras do VAE e amostras do GFN (para o caso `vae`).  
  *Nota: As referências ao WandB foram removidas.*
- **eval_step():** Realiza a avaliação do modelo, calculando métricas como o log-partition function e a log-verossimilhança média.
- **train_step(), fwd_train_step() e bwd_train_step():** Implementam os passos de treinamento (forward e backward).


In [6]:
def get_energy():
    if args.energy == 'vae':
        energy = VAEEnergy(device=device, batch_size=args.batch_size)
    elif args.energy == 'linreg':
        energy = LinearEnergy(device=device, batch_size=args.batch_size)
    else:
        raise NotImplementedError("Energia não implementada para este tipo.")
    return energy


def plot_step(energy, gfn_model, name):
    """
    Gera visualizações para o caso de energia 'vae' e salva os gráficos em arquivos PDF.
    As imagens não são enviadas para WandB.
    """
    if args.energy == 'vae':
        batch_size = plot_data_size
        real_data = energy.sample_evaluation_subset(batch_size)
        fig_real_data, ax_real_data = get_vae_images(real_data.detach().cpu())

        vae_samples_mu, vae_samples_logvar = energy.vae.encode(real_data)
        vae_z = energy.vae.reparameterize(vae_samples_mu, vae_samples_logvar)
        vae_samples = energy.vae.decode(vae_z)
        fig_vae_samples, ax_vae_samples = get_vae_images(vae_samples.detach().cpu())

        gfn_samples_z = gfn_model.sample(batch_size, energy.log_reward, real_data)
        gfn_samples = energy.vae.decode(gfn_samples_z)
        fig_gfn_samples, ax_gfn_samples = get_vae_images(gfn_samples.detach().cpu())

        fig_real_data.savefig(f'{name}real_data.pdf', bbox_inches='tight')
        fig_vae_samples.savefig(f'{name}vae_samples.pdf', bbox_inches='tight')
        fig_gfn_samples.savefig(f'{name}gfn_samples.pdf', bbox_inches='tight')

        # Retorna dicionário vazio (anteriormente, imagens eram enviadas via WandB)
        return {}
    else:
        return {}


def eval_step(eval_data, energy, gfn_model, final_eval=False, condition=None):
    """
    Avalia o modelo calculando métricas de log-partition function e log-verossimilhança média.
    """
    gfn_model.eval()
    metrics = dict()
    if final_eval:
        init_state = torch.zeros(final_eval_data_size, energy.data_ndim).to(device)
        samples, metrics['final_eval/log_Z'], metrics['final_eval/log_Z_lb'], metrics['final_eval/log_Z_learned'] = log_partition_function(
            init_state, gfn_model, energy.log_reward, condition=condition)
    else:
        init_state = torch.zeros(eval_data_size, energy.data_ndim).to(device)
        samples, metrics['eval/log_Z'], metrics['eval/log_Z_lb'], metrics['eval/log_Z_learned'] = log_partition_function(
            init_state, gfn_model, energy.log_reward, condition=condition)
    if eval_data is not None and condition is None:
        if final_eval:
            metrics['final_eval/mean_log_likelihood'] = mean_log_likelihood(eval_data, gfn_model, energy.log_reward,
                                                                            condition=condition)
        else:
            metrics['eval/mean_log_likelihood'] = mean_log_likelihood(eval_data, gfn_model, energy.log_reward,
                                                                      condition=condition)
        metrics.update(get_sample_metrics(samples, eval_data, final_eval))
    gfn_model.train()
    return metrics


def train_step(energy, gfn_model, gfn_optimizer, it, exploratory, epochs, buffer, buffer_ls, exploration_factor,
               exploration_wd, condition=None, repeats=10):
    gfn_model.zero_grad()
    exploration_std = get_exploration_std(it, exploratory, epochs, exploration_factor, exploration_wd)
    
    if args.both_ways:
        if it % 2 == 0:
            if args.sampling == 'buffer':
                loss, states, _, _, log_r = fwd_train_step(energy, gfn_model, exploration_std, return_exp=True,
                                                           condition=condition, repeats=repeats)

                states = states[:, -1]
                states = states.view(args.batch_size, repeats, -1)
                log_r = log_r.view(args.batch_size, repeats)
                states = states[torch.arange(args.batch_size), torch.argmax(log_r, dim=1)]
                log_r = log_r[torch.arange(args.batch_size), torch.argmax(log_r, dim=1)]
                buffer.add(states, log_r, condition=condition)
            else:
                loss = fwd_train_step(energy, gfn_model, exploration_std, condition=condition, repeats=repeats)
        else:
            loss = bwd_train_step(energy, gfn_model, buffer, buffer_ls, exploration_std, it=it, condition=condition,
                                  repeats=repeats)
    elif args.bwd:
        loss = bwd_train_step(energy, gfn_model, buffer, buffer_ls, exploration_std, it=it, condition=condition,
                              repeats=repeats)
    else:
        loss = fwd_train_step(energy, gfn_model, exploration_std, condition=condition, repeats=repeats)

    loss.backward()
    gfn_optimizer.step()
    return loss.item()


def fwd_train_step(energy, gfn_model, exploration_std, return_exp=False, condition=None, repeats=10):
    init_state = torch.zeros(args.batch_size, energy.data_ndim).to(device)
    loss = get_gfn_forward_loss(args.mode_fwd, init_state, gfn_model, energy.log_reward, coeff_matrix,
                                exploration_std=exploration_std, return_exp=return_exp, condition=condition,
                                repeats=repeats)
    return loss


def bwd_train_step(energy, gfn_model, buffer, buffer_ls, exploration_std=None, it=0, condition=None, repeats=10):
    if args.sampling == 'sleep_phase':
        samples = gfn_model.sleep_phase_sample(args.batch_size, exploration_std, condition=condition).to(device)
    elif args.sampling == 'energy':
        samples = energy.sample(args.batch_size).to(device)
    elif args.sampling == 'buffer':
        if args.local_search:
            if it % args.ls_cycle < 2:
                samples, _, condition, _ = buffer.sample()
                samples = samples.detach()
                condition = condition.detach()
                local_search_samples, log_r, condition = langevin_dynamics(samples, energy.log_reward, device, args,
                                                                           condition=condition)
                buffer_ls.add(local_search_samples.detach(), log_r.detach(), condition=condition)
            samples, log_r, condition, _ = buffer_ls.sample()
        else:
            samples, _, condition, _ = buffer.sample().detach()

    loss = get_gfn_backward_loss(args.mode_bwd, samples, gfn_model, energy.log_reward,
                                 exploration_std=exploration_std, condition=condition, repeats=repeats)
    return loss


# Treinamento de GFN: Pipeline Desmembrado

Esta sequência de células reproduz o pipeline de treinamento que estava encapsulado na função `train()`.  
Cada célula representa uma etapa do processo:
1. Preparação do ambiente e criação do diretório de salvamento.
2. Inicialização do modelo de energia e das amostras de avaliação.
3. Criação do modelo GFN e do otimizador (e, se configurado, o scheduler).
4. Inicialização dos buffers de replay.
5. Loop de treinamento com avaliação e salvamento periódico.
6. Avaliação final e salvamento do modelo final.

Os parâmetros são definidos previamente (por meio do objeto `args`) e as chamadas ao WandB foram removidas para execução local.

In [7]:
name = get_name(args)
if not os.path.exists(name):
    os.makedirs(name)

## 2. Inicialização do Modelo de Energia e Dados de Avaliação

Dependendo do tipo de energia configurado (por exemplo, `'vae'` ou `'linreg'`), amostramos os dados de avaliação.

In [8]:
energy = get_energy()
if args.energy in ['vae', 'linreg']:
    eval_data = energy.sample(eval_data_size, evaluation=True).to(device)
    final_eval_data = energy.sample(final_eval_data_size, evaluation=True).to(device)
else:
    eval_data = energy.sample(eval_data_size).to(device)
    final_eval_data = energy.sample(final_eval_data_size).to(device)

# Para registro interno (não utilizado para log externo)
config = args.__dict__
config["Experiment"] = f"{args.energy}"


## 3. Inicialização do Modelo GFN e do Otimizador

Aqui criamos o modelo GFN com os parâmetros definidos e configuramos o otimizador.  
Se o scheduler estiver ativado, ele é criado para ajustar a taxa de aprendizado durante o treinamento.


In [9]:
gfn_model = GFN(energy.data_ndim, args.s_emb_dim, args.hidden_dim, args.harmonics_dim, args.t_emb_dim,
                trajectory_length=args.T, clipping=args.clipping, lgv_clip=args.lgv_clip, gfn_clip=args.gfn_clip,
                langevin=args.langevin, learned_variance=args.learned_variance,
                partial_energy=args.partial_energy, log_var_range=args.log_var_range,
                pb_scale_range=args.pb_scale_range,
                t_scale=args.t_scale, langevin_scaling_per_dimension=args.langevin_scaling_per_dimension,
                conditional_flow_model=args.conditional_flow_model, learn_pb=args.learn_pb,
                pis_architectures=args.pis_architectures, lgv_layers=args.lgv_layers,
                joint_layers=args.joint_layers, zero_init=args.zero_init, device=device, energy=args.energy).to(device)

gfn_optimizer = get_gfn_optimizer(gfn_model, args.lr_policy, args.lr_flow, args.lr_back, args.learn_pb,
                                  args.conditional_flow_model, args.use_weight_decay, args.weight_decay,
                                  args.energy)

if args.scheduler:
    lambda_function = lambda iteration: 0.1 if iteration >= args.step_point else 1.0
    scheduler = torch.optim.lr_scheduler.LambdaLR(gfn_optimizer, lr_lambda=lambda_function)

print(gfn_model)
metrics = dict()

GFN(
  (t_model): TimeEncodingVAE(
    (t_model): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (s_model): StateEncodingVAE(
    (x_model): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): GELU(approximate='none')
      (2): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): GELU(approximate='none')
      )
      (3): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): GELU(approximate='none')
      )
      (4): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (d_model): DeepSet(
    (phi): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
    )
    (rho): Sequential(
      (0): Linear(in_features=64, out_fea

## 4. Inicialização dos Buffers de Replay

Cria-se o buffer padrão e um buffer específico para busca local (*local search*), se estiver configurado.


In [10]:
buffer = ReplayBuffer(args.buffer_size, device, energy.log_reward, args.batch_size,
                      data_ndim=energy.data_ndim, beta=args.beta, rank_weight=args.rank_weight,
                      prioritized=args.prioritized)
buffer_ls = ReplayBuffer(args.buffer_size, device, energy.log_reward, args.batch_size,
                         data_ndim=energy.data_ndim, beta=args.beta, rank_weight=args.rank_weight,
                         prioritized=args.prioritized)
gfn_model.train()

GFN(
  (t_model): TimeEncodingVAE(
    (t_model): Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (s_model): StateEncodingVAE(
    (x_model): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): GELU(approximate='none')
      (2): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): GELU(approximate='none')
      )
      (3): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): GELU(approximate='none')
      )
      (4): Linear(in_features=64, out_features=64, bias=True)
    )
  )
  (d_model): DeepSet(
    (phi): Sequential(
      (0): Linear(in_features=2, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
    )
    (rho): Sequential(
      (0): Linear(in_features=64, out_fea


## 5. Loop de Treinamento

Nesta célula executamos o loop de treinamento.  
Para cada iteração (época), são executados os seguintes passos:
- Definição da condição de amostragem (se aplicável).
- Execução de um passo de treinamento (forward ou backward) via `train_step`.
- Atualização do scheduler, caso esteja configurado.
- A cada 100 iterações, o modelo é avaliado e visualizações são geradas.
- A cada 1000 iterações, o estado do modelo é salvo.


In [11]:
for i in trange(args.epochs + 1):
    # Define a condição para a amostragem, se o tipo de energia for 'vae' ou 'linreg'
    if args.energy in ['vae', 'linreg']:
        condition = energy.sample(args.batch_size)
    else:
        condition = None

    metrics['train/loss'] = train_step(energy, gfn_model, gfn_optimizer, i, args.exploratory, args.epochs,
                                       buffer, buffer_ls, args.exploration_factor, args.exploration_wd,
                                       condition=condition, repeats=args.repeats)

    if args.scheduler:
        scheduler.step()

    if i % 100 == 0:
        if args.energy in ['vae', 'linreg']:
            condition = energy.sample(eval_data_size, evaluation=True)
        else:
            condition = None
        metrics.update(eval_step(eval_data, energy, gfn_model, final_eval=False, condition=condition))
        print('Epoch:', i, ' - log_Z_lb:', metrics.get('eval/log_Z_lb'))
        _ = plot_step(energy, gfn_model, name)

        if i % 1000 == 0:
            torch.save(gfn_model.state_dict(), f'{name}model.pt')
            torch.save({
                'epoch': i,
                'model_state_dict': gfn_model.state_dict(),
                'optimizer_state_dict': gfn_optimizer.state_dict(),
                'loss': metrics['train/loss'],
            }, f'{name}model.pt')

  0%|          | 1/25001 [00:01<8:53:34,  1.28s/it]

Epoch: 0  - log_Z_lb: tensor(-52.2279)


  0%|          | 6/25001 [00:05<5:56:55,  1.17it/s]


KeyboardInterrupt: 


## 6. Avaliação Final e Salvamento do Modelo

Após o término do treinamento, realizamos uma avaliação final do modelo e salvamos o estado final.


In [ ]:
if args.energy in ['vae', 'linreg']:
    condition = energy.sample(eval_data_size, evaluation=True)
else:
    condition = None

eval_results = eval_step(final_eval_data, energy, gfn_model, final_eval=True, condition=condition)
metrics.update(eval_results)
if 'tb-avg' in args.mode_fwd or 'tb-avg' in args.mode_bwd:
    if 'final_eval/log_Z_learned' in metrics:
        del metrics['final_eval/log_Z_learned']

torch.save({
    'epoch': i,
    'model_state_dict': gfn_model.state_dict(),
    'optimizer_state_dict': gfn_optimizer.state_dict(),
    'loss': metrics['train/loss'],
}, f'{name}model_final.pt')